# CycleGAN with Ignite and Nvidia/Apex

In this notebook we provide an implementation of [CycleGAN](https://arxiv.org/abs/1703.10593) and its automatic mixed precision training on "Horse 2 Zebra" dataset using Ignite and Nvidia/Apex.

**Update (05/2020):** thanks to **@mcarilli** a bug in PyTorch (https://github.com/pytorch/pytorch/issues/37157) with `ConvTranspose2d` was fixed since v1.6.0 (or master at the moment of writing) and this enables CycleGAN automatic mixed precision training.

### CycleGAN in a Nutshell

CycleGAN is unpaired image-to-image translation task from $A$ to $B$ and represented by two generative networks $G$ and $F$:
$$
\hat{y} = G(x) \in B,\text{ for } x \in A \\
\hat{x} = F(y) \in A,\text{ for } y \in B
$$

and two discriminators $D_A$ and $D_B$. Training of the networks is done by minimizing the loss is a sum of 3 components:
$$
\mathcal{L}(G, F, D_A, D_B) = \mathcal{L}_{GAN}(G, D_B) + \mathcal{L}_{GAN}(F, D_A) + \lambda \mathcal{L}_{cyc}(G, F)
$$
with GAN loss:
$$
\mathcal{L}_{GAN}(G, D_B) = \text{mean}_{x \in A}\left[ (D_B(G(x)) - 1)^2 \right]+ \text{mean}_{y \in B}\left[ (D_B(y) - 1)^2 \right] \\
\mathcal{L}_{GAN}(F, D_A) = \text{mean}_{y \in B}\left[ (D_A(F(y)) - 1)^2 \right]+ \text{mean}_{x \in A}\left[ (D_A(x) - 1)^2 \right]
$$
and forward and backward cycle consistency term:
$$
\mathcal{L}_{cyc}(G, F) = \text{mean}_{x \in A}\left[ |F(G(x)) - x|_1 \right] + \text{mean}_{y \in B}\left[ |G(F(y)) - y|_1 \right]
$$

Optionally, one can add identity loss terms. See [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/qa.md#what-is-the-identity-loss-322-373-362).

In [ ]:
!nvidia-smi

## Requirements

1) Let's download the dataset:

In [ ]:
!wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip -O/tmp/horse2zebra.zip
!7z x /tmp/horse2zebra.zip -o/tmp/

2.a) We need install PyTorch >= 1.6.0 and torchvision

In [ ]:
!pip install --pre --upgrade torch torchvision -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html

In [ ]:
import torch
torch.__version__

2.b) Next we need to install [nvidia/apex](https://github.com/NVIDIA/apex)

In [ ]:
# Install Apex:
# If torch cuda version and nvcc version match:
!pip install --upgrade --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" git+https://github.com/NVIDIA/apex/
# if above command is failing, please install apex without c++/cuda extensions:
# !pip install --upgrade --no-cache-dir git+https://github.com/NVIDIA/apex/

3) Install latest tensorboardX and nightly `pytorch-ignite`

In [ ]:
!pip install --pre --upgrade pytorch-ignite tensorboardX

In [ ]:
import ignite
ignite.__version__

In [ ]:
import random
import torch

seed = 17
random.seed(seed)
_ = torch.manual_seed(seed)

## Dataflow

Let's setup the datase

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class FilesDataset(Dataset):
    
    def __init__(self, path, extension="*.jpg"):
        self.path = Path(path)
        assert self.path.exists(), "Path '{}' is not found".format(path)
        self.images = list(self.path.rglob(extension))
        assert len(self.images) > 0, "No images with extension {} found at '{}'".format(extension, path)
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, i):
        return Image.open(self.images[i]).convert('RGB')

In [ ]:
from pathlib import Path

root = Path("/tmp/horse2zebra")

train_A = FilesDataset(root / "trainA")
train_B = FilesDataset(root / "trainB")

test_A = FilesDataset(root / "testA") 
test_B = FilesDataset(root / "testB")

Some details on the datasets:

In [ ]:
print("Dataset sizes: \ntrain A: {} | B: {}\ntest A: {} | B: {}\n\t".format(len(train_A), len(train_B), len(test_A), len(test_B)))

In [ ]:
print(f"Train random image sizes (A): {train_A[0].size}, {train_A[1].size}, {train_A[10].size}, {train_A[-1].size}")

In [ ]:
print(f"Train random image sizes (B): {train_B[0].size}, {train_B[1].size}, {train_B[10].size}, {train_B[-1].size}")

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title("Train dataset 'Horses'")
plt.imshow(train_A[10])
plt.subplot(122)
plt.title("Train dataset 'Zebras'")
plt.imshow(train_B[10])

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title("Test dataset 'Horses'")
plt.imshow(test_A[0])
plt.subplot(122)
plt.title("Test dataset 'Zebras'")
plt.imshow(test_B[0])

Let's create a dataset composed of random image pairs of datasets A and B

In [ ]:
import random


class Image2ImageDataset(Dataset):
    
    def __init__(self, ds_a, ds_b):
        self.dataset_a = ds_a
        self.dataset_b = ds_b
    
    def __len__(self):
        return max(len(self.dataset_a), len(self.dataset_b))

    def __getitem__(self, i):
        dp_a = self.dataset_a[i % len(self.dataset_a)]
        j = random.randint(0, len(self.dataset_b) - 1)
        dp_b = self.dataset_b[j]
        return {
            'A': dp_a,
            'B': dp_b
        }


class TransformedDataset(Dataset):
        
    def __init__(self, ds, transform):
        self.dataset = ds
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, i):
        return {k: self.transform(v) for k, v in self.dataset[i].items()}

In [ ]:
train_ab_ds = Image2ImageDataset(train_A, train_B)
test_ab_ds = Image2ImageDataset(test_A, test_B)

In [ ]:
dp = train_ab_ds[20]

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title("Train dataset 'Horses'")
plt.imshow(dp['A'])
plt.subplot(122)
plt.title("Train dataset 'Zebras'")
plt.imshow(dp['B'])

In [ ]:
dp = test_ab_ds[20]

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title("Test dataset 'Horses'")
plt.imshow(dp['A'])
plt.subplot(122)
plt.title("Test dataset 'Zebras'")
plt.imshow(dp['B'])

In [ ]:
from torchvision.transforms import Compose, ColorJitter, RandomHorizontalFlip, ToTensor, Normalize, RandomCrop

# To accelerate the training we reduce the image size to 200x200 pix instead of 256x256
train_transform = Compose([
    RandomCrop(200),
    RandomHorizontalFlip(),
    ColorJitter(),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
transformed_train_ab_ds = TransformedDataset(train_ab_ds, transform=train_transform)

# Please select appropriate batch_size value according to your infrastructure
batch_size = 10
train_ab_loader = DataLoader(transformed_train_ab_ds, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)


test_transform = Compose([
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
transformed_test_ab_ds = TransformedDataset(test_ab_ds, transform=test_transform)
batch_size = 10
test_ab_loader = DataLoader(transformed_test_ab_ds, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)

In [ ]:
import torchvision.utils as vutils

# Plot some training images
real_batch = next(iter(train_ab_loader))

plt.figure(figsize=(16, 8))
plt.axis("off")
plt.title("Training Images from A")
plt.imshow( 
    vutils.make_grid(real_batch['A'][:64], padding=2, normalize=True).cpu().numpy().transpose((1, 2, 0))
)

plt.figure(figsize=(16, 8))
plt.axis("off")
plt.title("Training Images from B")
plt.imshow(
    vutils.make_grid(real_batch['B'][:64], padding=2, normalize=True).cpu().numpy().transpose((1, 2, 0))
)
real_batch = None
torch.cuda.empty_cache()

## Generator and Discriminator networks

- Generator network architecture contains 9 residual blocks
Using paper's notations:
```
c7s1-64,d128,d256,R256,R256,R256,R256,R256,R256,R256,R256,R256,u128,u64,c7s1-3,tanh
```
where `c7s1-k` denotes a 7x7 Convolution-InstanceNorm-ReLU layer with `k` filters and stride 1. `dk` denotes a 3x3 Convolution-InstanceNorm-ReLU layer  with `k` filters and stride 2. Reflection padding was used to reduce artifacts. `Rk` denotes a residual block that contains two 3x3 convolutional layers with the same number of filters on both layer. `uk` denotes a 3x3 fractional-strided-Convolution-InstanceNorm-ReLU layer with `k` filters and stride 1/2.

In [ ]:
import torch
import torch.nn as nn


def get_conv_inorm_relu(in_planes, out_planes, kernel_size, stride, reflection_pad=True, with_relu=True):
    layers = []
    padding = (kernel_size - 1) // 2
    if reflection_pad:
        layers.append(nn.ReflectionPad2d(padding=padding))
        padding = 0
    layers += [
        nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.InstanceNorm2d(out_planes, affine=False, track_running_stats=False),
    ]
    if with_relu:
        layers.append(nn.ReLU(inplace=True))
    return nn.Sequential(*layers)


def get_conv_transposed_inorm_relu(in_planes, out_planes, kernel_size, stride):
    return nn.Sequential(
        nn.ConvTranspose2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=1, output_padding=1),
        nn.InstanceNorm2d(out_planes, affine=False, track_running_stats=False),
        nn.ReLU(inplace=True)
    )


class ResidualBlock(nn.Module):
    
    def __init__(self, in_planes):
        super(ResidualBlock, self).__init__()
        self.conv1 = get_conv_inorm_relu(in_planes, in_planes, kernel_size=3, stride=1)
        self.conv2 = get_conv_inorm_relu(in_planes, in_planes, kernel_size=3, stride=1, with_relu=False)        

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.conv2(x)        
        return x + residual


class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()
        
        self.c7s1_64 = get_conv_inorm_relu(3, 64, kernel_size=7, stride=1)
        self.d128 = get_conv_inorm_relu(64, 128, kernel_size=3, stride=2, reflection_pad=False)
        self.d256 = get_conv_inorm_relu(128, 256, kernel_size=3, stride=2, reflection_pad=False)

        self.resnet9 = nn.Sequential(*[ResidualBlock(256) for i in range(9)])

        self.u128 = get_conv_transposed_inorm_relu(256, 128, kernel_size=3, stride=2)
        self.u64 = get_conv_transposed_inorm_relu(128, 64, kernel_size=3, stride=2)
        self.c7s1_3 = get_conv_inorm_relu(64, 3, kernel_size=7, stride=1, with_relu=False)
        # Replace instance norm by tanh activation
        self.c7s1_3[-1] = nn.Tanh()

    def forward(self, x):
        # Encoding
        x = self.c7s1_64(x)
        x = self.d128(x)
        x = self.d256(x)
        
        # 9 residual blocks
        x = self.resnet9(x)

        # Decoding
        x = self.u128(x)
        x = self.u64(x)
        y = self.c7s1_3(x)
        return y


Let's check the network:

In [ ]:
x = torch.rand(4, 3, 256, 256)
g = Generator()
y = g(x)
y.shape

- Discriminator network is a PatchGAN (with receptive field 70x70):
```
C64-C128-C256-C512
```
where `Ck` denote a 4x4 Convolution-InstanceNorm-LeakyReLU layer with `k` filters and stride 2. After the last layer, 
a convolution to produce a 1-dimensional output is applied. No `InstanceNorm` for the first `C64` layer. Leaky ReLUs are with a slope of `0.2`. 

Good explanation is given in [this comment](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/issues/39#issuecomment-305575964) on what is a PatchGAN:
```
In fact, a "PatchGAN" is just a convnet! Or you could say all convnets are patchnets: the power of convnets is that they process each image patch identically and independently, which makes things very cheap (# params, time, memory), and, amazingly, turns out to work.

The difference between a PatchGAN and regular GAN discriminator is that rather the regular GAN maps from a 256x256 image to a single scalar output, which signifies "real" or "fake", whereas the PatchGAN maps from 256x256 to an NxN array of outputs X, where each X_ij signifies whether the patch ij in the image is real or fake. Which is patch ij in the input? Well, output X_ij is just a neuron in a convnet, and we can trace back its receptive field to see which input pixels it is sensitive to. In the CycleGAN architecture, the receptive fields of the discriminator turn out to be 70x70 patches in the input image!

This is all mathematically equivalent to if we had manually chopped up the image into 70x70 overlapping patches, run a regular discriminator over each patch, and averaged the results.

Maybe it would have been better if we called it a "Fully Convolutional GAN" like in FCNs... it's the same idea :)
```

In [ ]:
def get_conv_inorm_lrelu(in_planes, out_planes, stride=2, negative_slope=0.2):
    return nn.Sequential(
        nn.Conv2d(in_planes, out_planes, kernel_size=4, stride=stride, padding=1),
        nn.InstanceNorm2d(out_planes, affine=False, track_running_stats=False),
        nn.LeakyReLU(negative_slope=negative_slope, inplace=True)
    )


class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()
        self.c64 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.c128 = get_conv_inorm_lrelu(64, 128)
        self.c256 = get_conv_inorm_lrelu(128, 256)
        self.c512 = get_conv_inorm_lrelu(256, 512, stride=1)
        self.last_conv = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)

    def forward(self, x):
        x = self.c64(x)
        x = self.relu(x)

        x = self.c128(x)
        x = self.c256(x)
        x = self.c512(x)
        y = self.last_conv(x)
        return y


Let's check the network:

In [ ]:
x = torch.rand(4, 3, 256, 256)
d = Discriminator()
y = d(x)
y.shape

According to the paper, the weights are initialized from a Gaussian distribution $\mathcal{N}(0,0.02)$

In [ ]:
def init_weights(module):
    assert isinstance(module, nn.Module)
    if hasattr(module, "weight") and module.weight is not None:
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    if hasattr(module, "bias") and module.bias is not None:
        torch.nn.init.constant_(module.bias, 0.0)
    for c in module.children():
        init_weights(c)

In [ ]:
g = None; d = None

## Networks' automatic mixed precision training

In [ ]:
assert torch.backends.cudnn.enabled, "NVIDIA/Apex:Amp requires cudnn backend to be enabled."
torch.backends.cudnn.benchmark = True

In [ ]:
device = "cuda"

generator_A2B = Generator().to(device)
init_weights(generator_A2B)

discriminator_B = Discriminator().to(device)
init_weights(discriminator_B)

generator_B2A = Generator().to(device)
init_weights(generator_B2A)
discriminator_A = Discriminator().to(device)
init_weights(discriminator_A)

Similarly, we train the networks from scratch, with a learning rate of `0.0002`.

In [ ]:
from itertools import chain
import torch.optim as optim

lr = 0.0002
beta1 = 0.5

optimizer_G = optim.Adam(chain(generator_A2B.parameters(), generator_B2A.parameters()), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(chain(discriminator_A.parameters(), discriminator_B.parameters()), lr=lr, betas=(beta1, 0.999))

Let's define some helper functions:
- to turn on/off gradients

In [ ]:
def toggle_grad(model, on_or_off):
    # https://github.com/ajbrock/BigGAN-PyTorch/blob/master/utils.py#L674
    for param in model.parameters():
        param.requires_grad = on_or_off


try:
    from apex import amp
except ImportError:
    raise ImportError("Please install apex from https://www.github.com/nvidia/apex to run this example.")


# Initialize Amp
models, optimizers = amp.initialize([generator_A2B, generator_B2A, discriminator_A, discriminator_B], 
                                    [optimizer_G, optimizer_D],
                                    opt_level="O2", num_losses=2)

generator_A2B, generator_B2A, discriminator_A, discriminator_B = models
optimizer_G, optimizer_D = optimizers

### Fake images buffer trick

According to the paper, to reduce model oscillation and the discriminators are updated using a history of generated images rather than the ones produced by the latest generators. There is an image buffer that stores the X previously created images.

In [ ]:
buffer_size = 50
fake_a_buffer = []
fake_b_buffer = []


def buffer_insert_and_get(buffer, batch):
    output_batch = []
    for b in batch:
        b = b.unsqueeze(0)
        # if buffer is not fully filled:
        if len(buffer) < buffer_size:
            output_batch.append(b)
            buffer.append(b.cpu())
        elif random.uniform(0, 1) > 0.5:
            # Add newly created image into the buffer and put ont from the buffer into the output
            random_index = random.randint(0, buffer_size - 1)            
            output_batch.append(buffer[random_index].clone().to(device))
            buffer[random_index] = b.cpu()
        else:
            output_batch.append(b)
    return torch.cat(output_batch, dim=0)

Next, let's define a single iteration via `update_fn` function. This function is then used by `ignite.engine.Engine` to update models while running over the input data.

As suggested, we divide the objective by 2 while optimizing D, which slows down the rate at which D learns, relative to the rate of G. 

According to the paper:
- generator A is trained  minimize $\text{mean}_{x \in A}[(D_B(G(x)) − 1)^2]$ and cycle loss $\text{mean}_{x \in A}\left[ |F(G(x)) - x|_1 \right]$
- generator B is trained  minimize $\text{mean}_{y \in B}[(D_A(F(y)) − 1)^2]$ and cycle loss $\text{mean}_{y \in B}\left[ |G(F(y)) - y|_1 \right]$
- discriminators A is trained to minimize $\text{mean}_{x \in A}[(D_A(x) − 1)^2] + \text{mean}_{y \in B}[D_A(F(y))^2]$.
- discriminator B is trained to minimize $\text{mean}_{y \in B}[(D_B(y) − 1)^2] + \text{mean}_{x \in A}[D_B(G(x))^2]$.

In [ ]:
from ignite.utils import convert_tensor
import torch.nn.functional as F


lambda_value = 10.0


def discriminator_forward_pass(discriminator, batch_real, batch_fake, fake_buffer):
    decision_real = discriminator(batch_real)
    batch_fake = buffer_insert_and_get(fake_buffer, batch_fake)      
    decision_fake = discriminator(batch_fake)
    return decision_real, decision_fake


def compute_loss_generator(batch_decision, batch_real, batch_rec, lambda_value):
    # loss gan
    target = torch.ones_like(batch_decision)
    loss_gan = F.mse_loss(batch_decision, target)
    # loss cycle
    loss_cycle = F.l1_loss(batch_rec, batch_real) * lambda_value    
    return loss_gan + loss_cycle


def compute_loss_discriminator(decision_real, decision_fake):
    # loss = mean (D_b(y) − 1)^2 + mean D_b(G(x))^2    
    loss = F.mse_loss(decision_fake, torch.zeros_like(decision_fake))
    loss += F.mse_loss(decision_real, torch.ones_like(decision_real))
    return loss


def update_fn(engine, batch):
    generator_A2B.train()
    generator_B2A.train()
    discriminator_A.train()
    discriminator_B.train()

    real_a = convert_tensor(batch['A'], device=device, non_blocking=True)
    real_b = convert_tensor(batch['B'], device=device, non_blocking=True)
    
    # Update generators:

    # Disable grads computation for the discriminators:
    toggle_grad(discriminator_A, False)
    toggle_grad(discriminator_B, False)    
        
    fake_b = generator_A2B(real_a)
    rec_a = generator_B2A(fake_b)
    fake_a = generator_B2A(real_b)
    rec_b = generator_A2B(fake_a)
    decision_fake_a = discriminator_A(fake_a)
    decision_fake_b = discriminator_B(fake_b)

    # Compute loss for generators and update generators
    # loss_a2b = GAN loss: mean (D_B(G(x)) − 1)^2 + Forward cycle loss: || F(G(x)) - x ||_1    
    loss_a2b = compute_loss_generator(decision_fake_b, real_a, rec_a, lambda_value)    

    # loss_b2a = GAN loss: mean (D_A(F(x)) − 1)^2 + Backward cycle loss: || G(F(y)) - y ||_1
    loss_b2a = compute_loss_generator(decision_fake_a, real_b, rec_b, lambda_value)

    # total generators loss:
    loss_generators = loss_a2b + loss_b2a

    optimizer_G.zero_grad()    
    with amp.scale_loss(loss_generators, optimizer_G, loss_id=0) as scaled_loss:
        scaled_loss.backward()
    optimizer_G.step()

    decision_fake_a = rec_a = decision_fake_b = rec_b = None
    
    # Update discriminators:

    # Enable grads computation for the discriminators:
    toggle_grad(discriminator_A, True)
    toggle_grad(discriminator_B, True)

    decision_real_a, decision_fake_a = discriminator_forward_pass(discriminator_A, real_a, fake_a.detach(), fake_a_buffer)    
    decision_real_b, decision_fake_b = discriminator_forward_pass(discriminator_B, real_b, fake_b.detach(), fake_b_buffer)    
    # Compute loss for discriminators and update discriminators
    # loss_a = mean (D_a(y) − 1)^2 + mean D_a(F(x))^2
    loss_a = compute_loss_discriminator(decision_real_a, decision_fake_a)

    # loss_b = mean (D_b(y) − 1)^2 + mean D_b(G(x))^2
    loss_b = compute_loss_discriminator(decision_real_b, decision_fake_b)
    
    # total discriminators loss:
    loss_discriminators = 0.5 * (loss_a + loss_b)
    
    optimizer_D.zero_grad()
    with amp.scale_loss(loss_discriminators, optimizer_D, loss_id=1) as scaled_loss:
        scaled_loss.backward()
    optimizer_D.step()
    
    return {
        "loss_generators": loss_generators.item(),
        "loss_generator_a2b": loss_a2b.item(),
        "loss_generator_b2a": loss_b2a.item(),
        "loss_discriminators": loss_discriminators.item(),
        "loss_discriminator_a": loss_a.item(),
        "loss_discriminator_b": loss_b.item(),
    }
    

Let's check `update_fn`

In [ ]:
real_batch = next(iter(train_ab_loader))

res = update_fn(engine=None, batch=real_batch)

real_batch = None
torch.cuda.empty_cache()

res

Now let's define a trainer and add some practical handlers:
- log to tensorboard: losses, lr, generated images
- progress bar
- models/optimizers checkpointing


Optionally, we also log to online platform [W&B](https://app.wandb.ai). Current logs can be seen at https://app.wandb.ai/vfdev-5/ignite-cyclegan-apex

```
pip install --upgrade wandb
wandb login your-token
```

In [ ]:
!pip install --upgrade wandb
# !wandb login your-token

In [ ]:
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage

from ignite.handlers import TensorboardLogger, WandBLogger
from ignite.handlers.tensorboard_logger import OutputHandler, OptimizerParamsHandler

In [ ]:
from functools import partial


trainer = Engine(update_fn)

metric_names = [
    'loss_discriminators', 
    'loss_generators', 
    'loss_discriminator_a',
    'loss_discriminator_b',
    'loss_generator_a2b',
    'loss_generator_b2a'    
]

def output_transform(out, name):
    return out[name]

for name in metric_names:
    # here we cannot use lambdas as they do not store argument `name`
    RunningAverage(output_transform=partial(output_transform, name=name)).attach(trainer, name)


In [ ]:
from datetime import datetime

exp_name = datetime.now().strftime("%Y%m%d-%H%M%S")
tb_logger = TensorboardLogger(log_dir="/tmp/cycle_gan_horse2zebra_tb_logs/{}".format(exp_name))

tb_logger.attach(trainer, 
                 log_handler=OutputHandler('training', metric_names), 
                 event_name=Events.ITERATION_COMPLETED)

print("Experiment name: ", exp_name)

In [ ]:
from pathlib import Path

try:

    wb_run_name = "cycle_gan_horse2zebra"
    wb_dir = Path("/tmp/cycle_gan_horse2zebra_wandb")
    if not wb_dir.exists():
        wb_dir.mkdir()
    wb_logger = WandBLogger(
        project="ignite-cyclegan-apex",
        name=wb_run_name,
        sync_tensorboard=True,
        dir=wb_dir.as_posix(),
        reinit=True
    )
except RuntimeError:
    wb_logger = None

Let's create an evaluator to inference on train/test images and log the generated images to Tensorboard:

In [ ]:
from ignite.engine import Engine


def evaluate_fn(engine, batch):
    generator_A2B.eval()
    generator_B2A.eval()    
    with torch.no_grad():
        real_a = convert_tensor(batch['A'], device=device, non_blocking=True)
        real_b = convert_tensor(batch['B'], device=device, non_blocking=True)
        
        fake_b = generator_A2B(real_a)
        rec_a = generator_B2A(fake_b)

        fake_a = generator_B2A(real_b)
        rec_b = generator_A2B(fake_a)
        
    return {
        'real_a': real_a,
        'real_b': real_b,
        'fake_a': fake_a,
        'fake_b': fake_b,
        'rec_a': rec_a,
        'rec_b': rec_b,        
    }


evaluator = Engine(evaluate_fn)

In [ ]:
from torch.utils.data import Subset

eval_batch_size = 5

train_random_indices = [random.randint(0, len(train_ab_ds) - 1) for _ in range(eval_batch_size)]
small_train_ds = Subset(train_ab_ds, train_random_indices)
small_train_ds = TransformedDataset(small_train_ds, transform=test_transform)

test_random_indices = [random.randint(0, len(test_ab_ds) - 1) for _ in range(eval_batch_size)]
small_test_ds = Subset(test_ab_ds, test_random_indices)
small_test_ds = TransformedDataset(small_test_ds, transform=test_transform)

eval_train_loader = DataLoader(small_train_ds, batch_size=eval_batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)
eval_test_loader = DataLoader(small_test_ds, batch_size=eval_batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)

In [ ]:
@trainer.on(Events.EPOCH_STARTED)
def run_evaluation(engine):
    evaluator.run(eval_train_loader)
    evaluator.run(eval_test_loader)


def log_generated_images(engine, logger, event_name):

    tag = "Train" if engine.state.dataloader == eval_train_loader else "Test"
    output = engine.state.output
    state = trainer.state
    global_step = state.get_event_attrib_value(event_name)

    # create a grid:
    # [real a1, real a2, ...]
    # [fake a1, fake a2, ...]
    # [rec a1, rec a2, ...]
    
    s = output['real_a'].shape[0]
    res_a = vutils.make_grid(torch.cat([
        output['real_a'],
        output['fake_b'],
        output['rec_a'],
    ]), padding=2, normalize=True, nrow=s).cpu()

    logger.writer.add_image(tag="{} Horses2Zebras (real, fake, rec)".format(tag), 
                            img_tensor=res_a, global_step=global_step, dataformats='CHW')

    s = output['real_b'].shape[0]
    res_b = vutils.make_grid(torch.cat([
        output['real_b'],
        output['fake_a'],
        output['rec_b'],
    ]), padding=2, normalize=True, nrow=s).cpu()
    logger.writer.add_image(tag="{} Zebras2Horses (real, fake, rec)".format(tag), 
                            img_tensor=res_b, global_step=global_step, dataformats='CHW')

    
tb_logger.attach(evaluator,
                 log_handler=log_generated_images, 
                 event_name=Events.COMPLETED)

We also follow suggested lr scheduling: the same learning rate for the first 100 epochs and linearly decay the rate to zero over the next 100 epochs.

In [ ]:
from ignite.handlers import PiecewiseLinear, ParamGroupScheduler

lr = 0.0002

milestones_values = [
    (0, lr),
    (100, lr),
    (200, 0.0)
]
gen_lr_scheduler = PiecewiseLinear(optimizer_D, param_name='lr', milestones_values=milestones_values)
desc_lr_scheduler = PiecewiseLinear(optimizer_G, param_name='lr', milestones_values=milestones_values)

lr_scheduler = ParamGroupScheduler([gen_lr_scheduler, desc_lr_scheduler], 
                                   names=['gen_lr_scheduler', 'desc_lr_scheduler'])

trainer.add_event_handler(Events.EPOCH_STARTED, lr_scheduler)


tb_logger.attach(trainer,
                 log_handler=OptimizerParamsHandler(optimizer_G, "lr"), 
                 event_name=Events.EPOCH_STARTED)

Let's setup models/optimizers checkpointing:

In [ ]:
from ignite.handlers import ModelCheckpoint, TerminateOnNan

In [ ]:
!rm -rf "/tmp/cycle_gan_checkpoints" 
!mkdir "/tmp/cycle_gan_checkpoints"

from ignite.handlers import ModelCheckpoint, TerminateOnNan


checkpoint_handler = ModelCheckpoint(dirname="/tmp/cycle_gan_checkpoints", filename_prefix="")

to_save = {
    "generator_A2B": generator_A2B,
    "discriminator_B": discriminator_B,
    "generator_B2A": generator_B2A,
    "discriminator_A": discriminator_A,
    
    "optimizer_G": optimizer_G,
    "optimizer_D": optimizer_D,
}

trainer.add_event_handler(Events.ITERATION_COMPLETED(every=500), checkpoint_handler, to_save)
trainer.add_event_handler(Events.ITERATION_COMPLETED, TerminateOnNan())

In [ ]:
from ignite.handlers import ProgressBar

# Iteration-wise progress bar
ProgressBar(bar_format="").attach(trainer)
# Epoch-wise progress bar with display of training losses
ProgressBar(persist=True, bar_format="").attach(trainer, metric_names=['loss_discriminators', 'loss_generators'], 
                                                event_name=Events.EPOCH_STARTED, closing_event_name=Events.COMPLETED)

In [ ]:
# Display in Firefox may not work properly. Use Chrome.
%load_ext tensorboard

%tensorboard --logdir=/tmp/cycle_gan_horse2zebra_tb_logs

In [ ]:
trainer.run(train_ab_loader, max_epochs=200)

In [ ]:
tb_logger.close()

### Inference with trained generator

Let's display saved checkpoint, load weights for "Horses to Zebras" generator and run an inference on a test image

In [ ]:
!ls /tmp/cycle_gan_checkpoints/

In [ ]:
checkpoint_path = "/tmp/cycle_gan_checkpoints/checkpoint_26500.pt"

# let's save this checkpoint to W&B
if wb_logger is not None:
    wb_logger.save(checkpoint_path)

In [ ]:
checkpoint_state_dict = torch.load(checkpoint_path)
generator_A2B.load_state_dict(checkpoint_state_dict["generator_A2B"])

In [ ]:
def normalize(x):
    vmin = x.min()
    vmax = x.max()
    x.clamp_(min=vmin, max=vmax)
    x.add_(-vmin).div_(vmax - vmin + 1e-5)
    return x

In [ ]:
i = random.randint(0, len(test_ab_ds) - 1)
img = test_ab_ds[i]['A']
x = test_transform(img)
x = x.unsqueeze(0).to(device)


with torch.no_grad():
    y_pred = generator_A2B(x)
    

img_pred = (255 * normalize(y_pred[0, ...])).cpu().numpy().transpose((1, 2, 0)).astype('uint8')

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.title("Horse")
plt.imshow(img)
plt.subplot(122)
plt.title("Generated zebra")
plt.imshow(img_pred)

Let's apply for fun our zebra-filter on an image with deep learning gurus:

In [ ]:
!wget https://www.kdnuggets.com/wp-content/uploads/photo.jpg -O/tmp/dl_durus.jpg

In [ ]:
from PIL import Image

img = Image.open("/tmp/dl_durus.jpg")
x = test_transform(img)
x = x.unsqueeze(0).to(device)


with torch.no_grad():
    y_pred = generator_A2B(x)


img_pred = (255 * normalize(y_pred[0, ...])).cpu().numpy().transpose((1, 2, 0)).astype('uint8')

In [ ]:
plt.figure(figsize=(15, 8))
plt.subplot(121)
plt.title("4 Deep-learning gurus")
plt.imshow(img)
plt.subplot(122)
plt.title("Zebras")
plt.imshow(img_pred)